<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto1/Spotify_year_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de trends do Spotify
O objetivo desse projeto é analisar os datasets do Spotify, [disponível no Kaggle](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks), para extrair informações como quais os gêneros mais escutados ao longo dos anos e suas caracteristicas musicais.

Dos datasets fornecidos, iremos utilizar o `data_o.csv` e o `tracks.csv`, cujo já detém os dados que precisamos, como genero, ano, popularidade, [features do spotify](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features), etc.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Coleta
Nesse primeiro momento iremos carregar o dataset do Kaggle em nosso notebook e analisar os dados enviados, para dá inicio ao pré processamento.

### Kaggle

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d yamaerenay/spotify-dataset-19212020-160k-tracks

 98% 191M/195M [00:02<00:00, 117MB/s]
100% 195M/195M [00:02<00:00, 98.1MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  spotify-dataset-19212020-160k-tracks.zip
replace artists.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: artists.csv             
  inflating: data_by_artist_o.csv    
  inflating: data_by_genres_o.csv    
  inflating: data_by_year_o.csv      
  inflating: data_o.csv              
  inflating: dict_artists.json       
  inflating: tracks.csv              


### Carregando o dataset

In [14]:
df_data = pd.read_csv('data_o.csv')
df_tracks = pd.read_csv('tracks.csv')
df_data_by_genres = pd.read_csv('data_by_genres_o.csv')
df_data_by_artist = pd.read_csv('data_by_artist_o.csv')
df_data_by_year = pd.read_csv('data_by_year_o.csv')

In [ ]:
print(f"DF - df_data:")
display(df_data.head())


DF - df_data:


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [ ]:
print(f"# DF - df_tracks:")
display(df_tracks.head())

# DF - df_tracks:


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [29]:
print(f"# DF - df_data_by_artist:")
display(df_data_by_artist.head())

# DF - df_data_by_artist:


,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7


In [28]:
print(f"# DF - df_data_by_year:")
display(df_data_by_year.head())

# DF - df_data_by_year:


,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0
3,1,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10
4,1,1925,0.962607,0.573863,184986.924460,0.278594,0.418297,0.237668,-14.146414,0.111918,115.521921,0.621929,2.604317,5


In [42]:
print(f"# DF - df_data_by_genre:")
display(df_data_by_genres.head())

# DF - df_data_by_genre:


,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7


# 2. Pré Processamento

Analisar dados que não façam sentindo
Explicar o que é tempo e pq retirar as linhas com o valor igual a zero

In [6]:
df_data.loc[df_data['tempo'] == 0]

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
508,0.0,1924,0.99600,['Ignacio Corsini'],0.0,133947,0.025800,0,2WTtJDfCUjAyCZHRSfCLDn,0.76800,5,0.0831,-24.894,1,La Mina del Ford - Remasterizado,0,1924-03-29,0.0,0.0
591,0.0,1924,0.99600,['Francisco Canaro'],0.0,170253,0.315000,0,5tQsQaMquCXBREb1FNymi7,0.95600,8,0.3750,-15.969,0,Oh Mujer Mujer - Remasterizado,0,1924-07-12,0.0,0.0
2578,0.0,1934,0.00244,['The Moors'],0.0,14629,0.801000,0,29IVdlO7CSlcoOIpOsamkO,0.00125,4,0.0503,-0.671,0,Santa Claus Is Coming To Town,0,1934,0.0,0.0
2764,0.0,1935,0.93600,['Orquesta Francisco Canaro'],0.0,211880,0.202000,0,1QXbWUSsLWPtHrbZ7H8o9i,0.01370,6,0.1200,-12.399,1,Tu Y Yo,4,1935-01-01,0.0,0.0
3042,0.0,1937,0.89000,['Billie Holiday'],0.0,159600,0.281000,0,09eDRV4nzC3eVX7tDEJUNb,0.00000,10,0.2470,-13.078,1,Back In Your Own Backyard - Take 1,17,1937,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169591,0.0,2015,0.46900,['$uicideBoy$'],0.0,153809,0.630000,1,00CutyaH2ljgLtM0nM5LLd,0.37500,2,0.1270,-6.707,1,Clouds As Witnesses,52,2015-03-03,0.0,0.0
169717,0.0,2016,0.12700,"['Rain Sounds', 'Rain Sounds & White Noise', '...",0.0,93452,0.000062,0,6HMSnRW8Htfd3YxdNgUF73,0.99900,9,0.7180,-18.685,1,Pouring Rain,54,2016-03-01,0.0,0.0
169935,0.0,2017,0.42000,['Granular'],0.0,180656,0.001720,0,03sJaTenPLeRaVUISW6pI5,0.39900,4,0.1100,-27.848,0,Pink Noise - BP 240 hz,61,2017-11-24,0.0,0.0
169968,0.0,2017,0.96200,['Fabricants'],0.0,175680,0.000020,0,4UYLGTJRGCt4KzP9r9Y9bH,0.02800,11,0.1370,-52.220,0,Brain Relaxation,57,2017-01-18,0.0,0.0


Explicar sobre duração

In [7]:
df_data.loc[df_data['duration_ms'] == 0]

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo


In [8]:
df_data.loc[df_data['year'] == 0]

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo


Retirada de colunas consideradas dummy no dataset, visto que não iremos utiliza-las

In [9]:
df_data.drop(['mode'], axis=1)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,China,72,2020-05-29,0.0881,105.029
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,Darkness,70,2020-01-17,0.3080,75.055


Verificar se existem generos vazios


In [39]:
df_data_by_artist.loc[df_data_by_artist['genres'].notna()]

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28675,[],麥志誠,0.512000,0.356000,198773.000000,0.306000,0.008970,0.108000,-10.119000,0.027700,150.049000,0.328000,35.000000,10,1,2
28676,"['c-pop', 'classic cantopop', 'classic mandopo...",黃品源,0.541000,0.578000,293840.000000,0.334000,0.000006,0.067500,-11.974000,0.026700,135.934000,0.243000,48.000000,9,0,2
28677,[],黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,23.000000,5,1,11
28678,"['chinese indie', 'chinese indie rock']",黑豹,0.381000,0.353000,316160.000000,0.686000,0.000000,0.056800,-9.103000,0.039500,200.341000,0.352000,35.000000,11,1,2


In [40]:
df = pd.merge(df_data, df_tracks, left_on='id', right_on='id', how='inner', suffixes=('_data','_by_artist'))

In [41]:
display(df)

,valence_data,year,acousticness_data,artists_data,danceability_data,duration_ms_data,energy_data,explicit_data,id,instrumentalness_data,key_data,liveness_data,loudness_data,mode_data,name_data,popularity_data,release_date_data,speechiness_data,tempo_data,name_by_artist,popularity_by_artist,duration_ms_by_artist,explicit_by_artist,artists_by_artist,id_artists,release_date_by_artist,danceability_by_artist,energy_by_artist,key_by_artist,loudness_by_artist,mode_by_artist,speechiness_by_artist,acousticness_by_artist,instrumentalness_by_artist,liveness_by_artist,valence_by_artist,tempo_by_artist,time_signature
0,0.127,1922,0.67400,['Uli'],0.645,126903,0.4450,0,35iwgR4jXetI318WEWsa1Q,0.744000,0,0.1510,-13.338,1,Carve,6,1922-02-22,0.4510,104.851,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.67400,0.744000,0.1510,0.127,104.851,3
1,0.457,1922,0.99400,['Ignacio Corsini'],0.434,181640,0.1770,0,07A5yehtSnoedViJAZkNnc,0.021800,1,0.2120,-21.180,1,Vivo para Quererte - Remasterizado,0,1922-03-21,0.0512,130.418,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.99400,0.021800,0.2120,0.457,130.418,5
2,0.397,1922,0.99500,['Ignacio Corsini'],0.321,176907,0.0946,0,08FmqUhxtyLTn6pAh6bk45,0.918000,7,0.1040,-27.961,1,El Prisionero - Remasterizado,0,1922-03-21,0.0504,169.980,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.99500,0.918000,0.1040,0.397,169.980,3
3,0.727,1922,0.96500,['Francis Marty'],0.510,134467,0.3550,0,0Dd9ImXtAtGwsmsAD69KZT,0.000000,4,0.1550,-12.833,1,La Butte Rouge,0,1922,0.1240,85.754,La Butte Rouge,0,134467,0,['Francis Marty'],['2nuMRGzeJ5jJEKlfS7rZ0W'],1922,0.510,0.3550,4,-12.833,1,0.1240,0.96500,0.000000,0.1550,0.727,85.754,5
4,0.654,1922,0.99300,['Mistinguett'],0.563,161427,0.1840,0,0IA0Hju8CAgYfV1hwhidBH,0.000016,4,0.3250,-13.757,1,La Java,0,1922,0.0512,133.088,La Java,0,161427,0,['Mistinguett'],['4AxgXfD7ISvJSTObqm4aIE'],1922,0.563,0.1840,4,-13.757,1,0.0512,0.99300,0.000016,0.3250,0.654,133.088,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126288,0.659,2020,0.00248,"['DVBBS', 'Quinn XCII']",0.714,150819,0.7980,0,4fCE1SCDulouoac9Q8XroU,0.000678,5,0.1030,-4.239,0,West Coast (feat. Quinn XCII),67,2020-10-16,0.0612,119.017,West Coast (feat. Quinn XCII),69,150819,0,"['DVBBS', 'Quinn XCII']","['5X4LWwbUFNzPkEas04uU82', '3ApUX1o6oSz321MMEC...",2020-10-16,0.714,0.7980,5,-4.239,0,0.0612,0.00248,0.000678,0.1030,0.659,119.017,4
126289,0.738,2020,0.16000,"['Rauw Alejandro', 'Dalex', 'Lenny Tavárez', '...",0.788,325692,0.6290,1,3oCghu20Kto5Xcu8Pg0cit,0.000828,0,0.0598,-4.727,1,"Elegí (feat. Farruko, Anuel AA, Sech, Dímelo F...",75,2020-08-28,0.0883,172.044,"Elegí (feat. Farruko, Anuel AA, Sech, Dímelo F...",68,325692,1,"['Rauw Alejandro', 'Dalex', 'Lenny Tavárez', '...","['1mcTU81TzQhprhouKaTkpq', '0KPX4Ucy9dk82uj4Gp...",2020-08-28,0.788,0.6290,0,-4.727,1,0.0883,0.16000,0.000828,0.0598,0.738,172.044,4
126290,0.394,2020,0.20100,"['Myke Towers', 'Maluma', 'Farruko']",0.775,250333,0.6330,1,70zg99pT51vB4wlMS7e4q7,0.000000,10,0.2190,-3.332,0,La Playa - Remix,74,2020-01-03,0.1330,140.008,La Playa - Remix,72,250333,1,"['Myke Towers', 'Maluma', 'Farruko']","['7iK8PXO48WeuP03g8YR51W', '1r4hJ1h58CWwUQe3Mx...",2020-01-03,0.775,0.6330,10,-3.332,0,0.1330,0.20100,0.000000,0.2190,0.394,140.008,4
126291,0.608,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.8080,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029,China,72,301714,0,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...","['2R21vXR83lH98kGeO99Y66', '4VMYDCV2IEDYJArk74...",2020-05-29,0.786,0.8080,7,-3.702,1,0.0881,0.08460,0.000289,0.0822,0.608,105.029,4
